In [2]:
import sys
sys.path.append('..')
from MPRA_predict.utils import *
from MPRA_predict.metrics import *

from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.neural_network import MLPRegressor, MLPClassifier
from sklearn.svm import SVR, SVC
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier, GradientBoostingRegressor, GradientBoostingClassifier

In [3]:
def find_indice(info_df, cell_types=[], assays=[], IDs=[]):
    # 初始化索引为全部行
    indice = list(range(len(info_df)))
    
    # 如果 cell_types 不为空，则筛选 cell_type
    if cell_types:
        indice = [i for i in indice if info_df['cell_type'][i] in cell_types]
        
    # 如果 assays 不为空，则筛选 assay
    if assays:
        indice = [i for i in indice if info_df['assay'][i] in assays]
        
    # 如果 IDs 不为空，则筛选 ID
    if IDs:
        indice = [i for i in indice if info_df['ID'][i] in IDs]
    
    return indice

In [4]:
info_df = pd.read_csv('../pretrained_models/Sei/Sei_info.csv')
info_df

FileNotFoundError: [Errno 2] No such file or directory: '../pretrained_models/Sei/Sei_info.csv'

In [4]:
# pretrained_pred = np.load('../pretrained_models/Sei/Sei_Agarwal_pred.npy')
# print(pretrained_pred.shape)

# main_df = pd.read_csv('/home/hxcai/cell_type_specific_CRE/data/AgarwalMPRA/Agarwal.csv')
# print(main_df.shape)

pretrained_pred = np.load('../pretrained_models/Sei/Sei_Siraj_pred.npy')
print(pretrained_pred.shape)

main_df = pd.read_csv('../../data/SirajMPRA/SirajMPRA_total.csv')
print(main_df.shape)

(585605, 21907)
(585605, 11)


In [5]:
# pretrained_pred = np.load('../pretrained_models/Sei/Sei_Siraj_pred.npy')
# print(pretrained_pred.shape)

# pred = np.load('../pretrained_models/Sei/Sei_Gosai_designed_pred.npy')
# pretrained_pred = np.concatenate([pretrained_pred, pred])
# print(pretrained_pred.shape)


# main_df = pd.read_csv('/home/hxcai/cell_type_specific_CRE/data/SirajMPRA/SirajMPRA_total.csv')
# print(main_df.shape)

# df = pd.read_csv('/home/hxcai/cell_type_specific_CRE/data/GosaiMPRA/GosaiMPRA_designed.csv')
# df['chr'] = 'chr0'
# main_df = pd.concat([main_df, df]).reset_index(drop=True)
# print(main_df.shape)

In [6]:
# # 不一定全
# cell_types_names_dict = {
#     'HepG2': ['HepG2', 'HepG2_Epithelium_Liver', 'HepG2_Hepatocellular_Carcinoma'],
#     'K562' : ['K562', 'K562_Erythroblast_Bone_Marrow'],
#     'SKNSH': ['SK-N-SH_RA_Neuron_Brain', 'Neuroblastoma_cell', 'SK-N-SH_Neuroblastoma_cell_Brain'],
#     'A549' : ['A549_Epithelium_Lung'],
#     'HCT116':['HCT-116_HCT116', 'HCT-116_Colon'],
#     'WTC11': ['iPSC'],
# }


cell_types_names_dict = {
    'HepG2': ['HepG2_Epithelium_Liver'],
    'K562' : ['K562_Erythroblast_Bone_Marrow'],
    'SKNSH': ['SK-N-SH_Neuroblastoma_cell_Brain'],
    'A549' : ['A549_Epithelium_Lung'],
    'HCT116':['HCT-116_Colon'],
    'WTC11': ['iPSC'],
}

cell_types = ['HepG2']

warnings.filterwarnings("ignore", category=RuntimeWarning)

for cell_type in cell_types:
    cell_names = cell_types_names_dict[cell_type]
    matched_indice = find_indice(info_df, cell_names)
    features = logit(pretrained_pred[:, matched_indice])
print(features.shape)

(585605, 289)


In [7]:
data_split_dict = {}
data_split_dict['total'] = np.ones(len(main_df), dtype=bool)
data_split_dict['train'] = main_df['chr'].isin([f'chr{i}' for i in range(3, 23)])
data_split_dict['valid'] = main_df['chr'].isin(['chr1'])
data_split_dict['test'] = main_df['chr'].isin(['chr2'])
data_split_dict['chrom'] = main_df['chr'].isin([f'chr{i}' for i in range(1, 23)])
data_split_dict['designed'] = main_df['chr'].isin(['chr0'])


cell_types = ['HepG2', 'K562', 'A549', 'HCT116']
main_df['diff_exp'] = (main_df[cell_types].max(axis=1) - main_df[cell_types].min(axis=1)) > 1
data_split_dict['const'] = ~main_df['diff_exp']
data_split_dict['diff']  = main_df['diff_exp']

109816


In [9]:
cell_types = ['HepG2']
data_splits = ['train', 'valid', 'test', 'chrom', 'designed']
data_splits_2 = ['total', 'const', 'diff']

r_matrix = np.zeros((len(cell_types), len(data_splits) + len(data_splits_2)))
r_matrix = pd.DataFrame(r_matrix, index=cell_types, columns=data_splits + data_splits_2)


for cell_type in cell_types:
    # train
    data_split = 'train'
    print(f'train model by {cell_type} {data_split} data')
    condition = data_split_dict[data_split]
    train_df = main_df[condition]
    train_features = features[condition]
    X_train = train_features
    y_train = np.array(train_df[f'{cell_type}'])
    print(f'X_train shape: {X_train.shape}, y_train shape: {y_train.shape}')

    X_train, y_train = remove_nan(X_train, y_train)
    if len(X_train) == 0:
        continue
    
    # model = LinearRegression()
    # model = Lasso(alpha=1e-3)
    model = MLPRegressor(hidden_layer_sizes=10)
    model.fit(X_train, y_train)

    # valid
    for cell_type in cell_types:
        for data_split in data_splits:
            condition = data_split_dict[data_split]
            valid_df = main_df[condition]
            valid_features = features[condition]
            X_valid = valid_features
            y_valid = np.array(valid_df[f'{cell_type}'])

            X_valid, y_valid = remove_nan(X_valid, y_valid)
            if len(X_valid) == 0:
                continue
            y_pred = model.predict(X_valid)
            r = pearson(y_valid, y_pred)
            r_matrix[data_split][cell_type] = r

        for data_split in data_splits_2:
            condition = data_split_dict[data_split]
            valid_df = main_df[condition]
            valid_features = features[condition]
            X_valid = valid_features
            y_valid = np.array(valid_df[f'{cell_type}'])

            X_valid, y_valid = remove_nan(X_valid, y_valid)
            if len(X_valid) == 0:
                continue
            y_pred = model.predict(X_valid)
            r = pearson(y_valid, y_pred)
            r_matrix[data_split][cell_type] = r

    print(r_matrix)
    print()

train model by HepG2 train data
X_train shape: (478173, 289), y_train shape: (478173,)


/home/hxcai/anaconda3/envs/torch/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:698: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


          train     valid      test     chrom  designed     total     const  \
HepG2  0.720143  0.706634  0.691887  0.716927       0.0  0.717103  0.606571   

           diff  
HepG2  0.713404  



In [8]:
cell_types = ['HepG2']
data_splits = ['train', 'valid', 'test', 'chrom', 'designed']
data_splits_2 = ['const', 'diff']

r_matrix = np.zeros((len(cell_types), len(data_splits) + len(data_splits_2)))
r_matrix = pd.DataFrame(r_matrix, index=cell_types, columns=data_splits + data_splits_2)


for cell_type in cell_types:
    # train
    data_split = 'train'
    print(f'train model by {cell_type} {data_split} data')
    condition = data_split_dict[data_split]
    train_df = main_df[condition]
    train_features = features[condition]
    X_train = train_features
    y_train = np.array(train_df[f'{cell_type}'])
    print(f'X_train shape: {X_train.shape}, y_train shape: {y_train.shape}')

    X_train, y_train = remove_nan(X_train, y_train)
    if len(X_train) == 0:
        continue
    
    # model = LinearRegression()
    # model = Lasso(alpha=1e-3)
    model = SVR(kernel='linear', C=0.1, epsilon=0.1)
    model.fit(X_train, y_train)

    # valid
    for cell_type in cell_types:
        for data_split in data_splits:
            condition = data_split_dict[data_split]
            valid_df = main_df[condition]
            valid_features = features[condition]
            X_valid = valid_features
            y_valid = np.array(valid_df[f'{cell_type}'])

            X_valid, y_valid = remove_nan(X_valid, y_valid)
            if len(X_valid) == 0:
                continue
            y_pred = model.predict(X_valid)
            r = pearson(y_valid, y_pred)
            r_matrix[data_split][cell_type] = r

        for data_split in data_splits_2:
            condition = data_split_dict[data_split]
            valid_df = main_df[condition]
            valid_features = features[condition]
            X_valid = valid_features
            y_valid = np.array(valid_df[f'{cell_type}'])

            X_valid, y_valid = remove_nan(X_valid, y_valid)
            if len(X_valid) == 0:
                continue
            y_pred = model.predict(X_valid)
            r = pearson(y_valid, y_pred)
            r_matrix[data_split][cell_type] = r

    print(r_matrix)
    print()

train model by HepG2 train data
X_train shape: (478173, 289), y_train shape: (478173,)


In [ ]:
cell_types = ['HepG2']
data_splits = ['train', 'valid', 'test', 'chrom', 'designed']
data_splits_2 = ['const', 'diff']

r_matrix = np.zeros((len(cell_types), len(data_splits) + len(data_splits_2)))
r_matrix = pd.DataFrame(r_matrix, index=cell_types, columns=data_splits + data_splits_2)


for cell_type in cell_types:
    # train
    data_split = 'train'
    print(f'train model by {cell_type} {data_split} data')
    condition = data_split_dict[data_split]
    train_df = main_df[condition]
    train_features = features[condition]
    X_train = train_features
    y_train = np.array(train_df[f'{cell_type}'])
    print(f'X_train shape: {X_train.shape}, y_train shape: {y_train.shape}')

    X_train, y_train = remove_nan(X_train, y_train)
    if len(X_train) == 0:
        continue
    
    model = LinearRegression()
    # model = Lasso(alpha=1e-3)
    # model = SVR()
    model.fit(X_train, y_train)

    # valid
    for cell_type in cell_types:
        for data_split in data_splits:
            condition = data_split_dict[data_split]
            valid_df = main_df[condition]
            valid_features = features[condition]
            X_valid = valid_features
            y_valid = np.array(valid_df[f'{cell_type}'])

            X_valid, y_valid = remove_nan(X_valid, y_valid)
            if len(X_valid) == 0:
                continue
            y_pred = model.predict(X_valid)
            r = pearson(y_valid, y_pred)
            r_matrix[data_split][cell_type] = r

        for data_split in data_splits_2:
            condition = data_split_dict[data_split]
            valid_df = main_df[condition]
            valid_features = features[condition]
            X_valid = valid_features
            y_valid = np.array(valid_df[f'{cell_type}'])

            X_valid, y_valid = remove_nan(X_valid, y_valid)
            if len(X_valid) == 0:
                continue
            y_pred = model.predict(X_valid)
            r = pearson(y_valid, y_pred)
            r_matrix[data_split][cell_type] = r

    print(r_matrix)
    print()

train model by HepG2 train data
X_train shape: (478173, 289), y_train shape: (478173,)
          train     valid      test     chrom  designed    const      diff
HepG2  0.729657  0.718528  0.702028  0.726704       0.0  0.62116  0.723601



In [ ]:
cell_types = ['HepG2']
data_splits = ['train', 'valid', 'test', 'chrom', 'designed']
data_splits_2 = ['const', 'diff']

r_matrix = np.zeros((len(cell_types), len(data_splits) + len(data_splits_2)))
r_matrix = pd.DataFrame(r_matrix, index=cell_types, columns=data_splits + data_splits_2)


for cell_type in cell_types:
    # train
    data_split = 'train'
    print(f'train model by {cell_type} {data_split} data')
    condition = data_split_dict[data_split]
    train_df = main_df[condition]
    train_features = features[condition]
    X_train = train_features
    y_train = np.array(train_df[f'{cell_type}'])
    print(f'X_train shape: {X_train.shape}, y_train shape: {y_train.shape}')

    X_train, y_train = remove_nan(X_train, y_train)
    if len(X_train) == 0:
        continue
    
    model = RandomForestRegressor(n_estimators=100)
    # model = SVR()
    model.fit(X_train, y_train)

    # valid
    for cell_type in cell_types:
        for data_split in data_splits:
            condition = data_split_dict[data_split]
            valid_df = main_df[condition]
            valid_features = features[condition]
            X_valid = valid_features
            y_valid = np.array(valid_df[f'{cell_type}'])

            X_valid, y_valid = remove_nan(X_valid, y_valid)
            if len(X_valid) == 0:
                continue
            y_pred = model.predict(X_valid)
            r = pearson(y_valid, y_pred)
            r_matrix[data_split][cell_type] = r

        for data_split in data_splits_2:
            condition = data_split_dict[data_split]
            valid_df = main_df[condition]
            valid_features = features[condition]
            X_valid = valid_features
            y_valid = np.array(valid_df[f'{cell_type}'])

            X_valid, y_valid = remove_nan(X_valid, y_valid)
            if len(X_valid) == 0:
                continue
            y_pred = model.predict(X_valid)
            r = pearson(y_valid, y_pred)
            r_matrix[data_split][cell_type] = r

    print(r_matrix)
    print()